# LLaMA-3 Tokenizer  
# 1. Import

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "allganize/Llama-3-Alpha-Ko-8B-Evo"

In [35]:
model_id = "allganize/Llama-3-Alpha-Ko-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 2. Encode & Decode

In [ ]:
llm_input = [
    # {"Card_Name":"삼성카드 & MILEAGE PLATINUM(스카이패스)","benefits":"3% 캐시백, 영화 할인"},
    {"Card_Name":"KB국민 와이즈카드", "benefits":"뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화, 교육/육아"}
]

# 3. Template  
- user : 사용자 신분 : 사용자가 챗봇에게 질문할 내용이 됩니다.  
- system : LLM에게 시키고 싶은 것 : LLM이 어떤 기준을 가지고 작동할지에 대한 설정  
- assistant : LLM의 답변  


In [ ]:
messages = [
    {"role": "system", "content":"당신은 광고 카피라이터입니다. 입력된 정보에 기반하여 기발하고 명확한 카피라이팅 문구를 제작하세요."},
    {"role" :"user", "content" : llm_input}
]

# 4. ChatTemplate 인코딩 & 디코딩

In [36]:
# ChatTemplate 만들기
template_messages = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors= 'pt' # 파이토치 텐서로 리턴
    )
print(template_messages)

tensor([[128000, 128006,   9125, 128007,    271,  65895,  83628,  34804, 112601,
         103236, 102477, 108157,  34961,  80052,     13,  43449,  53400,  61139,
          19954, 126470,  83290,  55216, 102133, 101360, 104167, 111372,  24486,
         103236, 102477, 108157, 105876,  54535,  89359,  18918, 114699,  92245,
             13, 128009, 128006,    882, 128007,    271,     58,  13922,   5889,
          19552,   1232,    364,  30962, 100654, 101607,  75984, 103618, 102668,
         101436,  30446,    518,    364,  68244,   1220,   1232,    364, 114448,
         102199,     11,  38295,    102,  64189,     11, 103236, 104249,     14,
         105010,  13094, 106153,  29102,     11, 121346, 113629,     14,  63375,
         104790,  89881,  38187,     11, 101787,  64356,     11, 101999, 102233,
             14,  55430, 101314,     11, 115762,     11, 109194,     14, 102946,
          54059,   8439,     60, 128009, 128006,  78191, 128007,    271]])


# 5. LLaMA-3 모델 호출하기

In [ ]:
# 비트 양자화를 위한 패키지입니다. 모델의 용량을 줄여주는 역할을 한다.
# 설치 후 세션 다시 시작
# !pip install -q -U bitsandbytes

In [37]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

print(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

allganize/Llama-3-Alpha-Ko-8B-Instruct


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [38]:
device = 'cuda'

# 모델에 입력할 데이터를 gpu로 옮긴다.
model_inputs = template_messages.to(device)
model_inputs

tensor([[128000, 128006,   9125, 128007,    271,  65895,  83628,  34804, 112601,
         103236, 102477, 108157,  34961,  80052,     13,  43449,  53400,  61139,
          19954, 126470,  83290,  55216, 102133, 101360, 104167, 111372,  24486,
         103236, 102477, 108157, 105876,  54535,  89359,  18918, 114699,  92245,
             13, 128009, 128006,    882, 128007,    271,     58,  13922,   5889,
          19552,   1232,    364,  30962, 100654, 101607,  75984, 103618, 102668,
         101436,  30446,    518,    364,  68244,   1220,   1232,    364, 114448,
         102199,     11,  38295,    102,  64189,     11, 103236, 104249,     14,
         105010,  13094, 106153,  29102,     11, 121346, 113629,     14,  63375,
         104790,  89881,  38187,     11, 101787,  64356,     11, 101999, 102233,
             14,  55430, 101314,     11, 115762,     11, 109194,     14, 102946,
          54059,   8439,     60, 128009, 128006,  78191, 128007,    271]],
       device='cuda:0')

In [39]:
# 답변 종료 토큰 설정
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

### **답변 생성하기** : LLM에서 답변을 받기 위해서는 generate 함수 사용

In [40]:
# 모델 입력
generated_ids = model.generate(
    model_inputs, # 정수 인코딩이 된 프롬프트(chat Template)
    max_new_tokens=300, # 모델이 대답할 응답 메시지의 최대 길이
    eos_token_id = terminators, # 모델의 텍스트 생성 종료 토큰
    repetition_penalty = 1.05 # 반복된 응답 지수. 값이 작아질 수록 반복된 응답을 수행
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [41]:
# batch_decode : 여러 개의 input에 대해 한꺼번에 변환
decoded = tokenizer.batch_decode(generated_ids)
decoded[0]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n당신은 광고 카피라이터입니다. 입력된 정보에 기반하여 기발하고 명확한 카피라이팅 문구를 제작하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n[{\'Card_Name\': \'KB국민 와이즈카드\', \'benefits\': \'뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화, 교육/육아\'}]<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n**"The Wiser Choice"**\n\n**"KB 국민 와이즈카드"**\n\n**"지금까지의 생활을 더 쉽고 편리하게"**\n\n**"뷰티와 건강을 위한 다양한 혜택"**\n- 매장에서 사용할 수 있는 포인트 적립\n- 납부와 결제가 간편해진다.\n\n**"일상 생활의 모든 순간을 즐기세요"**\n- 카페와 베이커리에서 커피 한 잔과 함께\n- 쇼핑과 간편결제로 시간을 절약\n- 의료와 교통, 주유, 문화, 교육, 육아 등 모든 분야에서 편리함을 누리세요.\n\n**"지금 바로 시작하세요"**\n- **KB 국민 와이즈카드**로 더 나은 삶을 누리세요!**\n\n이 카피는 고객에게 편리함과 혜택을 제공하는 카드의 주요 기능을 강조하면서도, 일상 생활에서의 다양한 경험을 통해 고객이 느끼는 만족감을 표현하고 있습니다. 또한, 카드의 브랜드인 **KB 국민 와이즈카드**의 이미지를 강화하기 위해 \'지금까지의 생활을 더 쉽고 편리하게\'라는 문구를 사용했습니다. <|end_of_text|>'

In [32]:
!pip -q install langchain pypdf chromadb sentence-transformers faiss-gpu langchain-community

# 파이프라인 정의  
- 랭체인에서 사용할 내용 : 허깅페이스 오픈 LLM을 연결할 파이프라인을 정의  
- 허깅페이스의 파이프라인을 생성해서 랭체인의 파이프라인으로 이어주기

In [44]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import pipeline # 오픈 모델 체이닝
from langchain.chains import LLMChain

# model_id = "allganize/Llama-3-Alpha-Ko-8B-Instruct"
# tokenizer = AutoTokenizer.from_pretrained(model_id)

# 1. 허깅페이스 파이프라인 정의. LLM은 생성 모델이기 때문에 text-generation 파이프라인을 생성
text_generation_pipeline = pipeline(
    model = model, # 사용할 모델. 여기서는 LLaMA-3 모델
    tokenizer = tokenizer, # LLaMA-3의 토크나이저
    task = 'text-generation',
    return_full_text=False, # 답변만 생성 결과로 받고 싶은 경우 사용. True로 설정하면 입력 프롬프트까지 모두 나옴
    max_new_tokens=300,
    temperature=0.7, # 응답의 창의성 조정
    top_p=0.9 # 생성 확률의 누적값 기반으로 다양성 제어
)


#  프롬프트 템플릿 직접 정의
prompt_template = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 광고 카피라이터입니다. 주어진 카드 정보와 혜택을 바탕으로 기발하고 명확한 카피라이팅 문구를 제작하는 것이 역할입니다.
당신의 문구는 사용자에게 카드의 가치를 한눈에 전달할 수 있어야 하며, 읽는 사람이 혜택을 즉각적으로 이해하고 매력을 느끼게 해야 합니다.
당신은 뛰어난 광고 카피라이터입니다. 주어진 카드 정보와 혜택을 바탕으로 창의적이고 감동적인 카피라이팅 문구를 제작하는 것이 역할입니다.
- 당신의 문구는 사용자에게 카드의 가치를 한눈에 전달할 수 있어야 하며, 읽는 사람이 혜택을 즉각적으로 이해하고 매력을 느끼게 해야 합니다.
- 당신의 문구는 단순한 정보 전달이 아니라, 감정적인 연결을 만들어내야 합니다.
- 사용자가 이 카드를 소유했을 때 느낄 수 있는 만족감을 생생하게 그려주세요.
- 구체적인 혜택을 중심으로 재미있고 톡톡 튀는 문구를 작성해주세요.

<|eot_id|><|start_header_id|>user<|end_header_id|>

카드 정보:
{llm_input}

요청사항:
입력된 카드 정보 각각에 대해 3개의 톡톡 튀는 광고 문구를 작성해주세요.
각 문구는 다음을 포함해야 합니다:
- 카드의 주요 혜택 강조
- 감성적 또는 실용적 요소
- 이모티콘으로 생동감 추가

예시 문구:
1. "커피 한 잔의 행복도 더! ☕ 카페 할인 혜택, 바로 신한카드 SIMPLE로!"
2. "여행 가방 속 필수템, 대한항공 마일리지 카드✈️로 세계를 담아보세요."
3. "월말 지출도 든든하게! 💳 삼성카드 SAVE로 캐시백의 즐거움!"

위와 같은 스타일로 작성해주세요.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

# 2. Langchain 파이프라인 정의
# 허깅페이스 파이프라인 사용
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# 3. Langchain의 프롬프트 템플릿 생성
prompt = PromptTemplate(
    input_variables=["llm_input"],
    template= prompt_template
)

# llm chain 구축
llm_chain = LLMChain(llm=llm, prompt=prompt)

llm_input = """
- 카드 이름: KB국민 와이즈카드
- 혜택: 뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화, 교육/육아
"""

# 실행
response = llm_chain.run({"llm_input":llm_input})


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [45]:
print(response)

다음은 카드 정보에 맞는 3개의 톡톡 튀는 광고 문구입니다.

1. "뷰티의 기쁨을 더해! 🎀 KB국민 와이즈카드, 뷰티, 납부, 카페/베이커리 혜택으로 여유로운 삶을 즐기세요."
2. "지출이 즐거워지는 순간! 🛒 쇼핑/간편결제, 의료, 교통/주유, 문화, 교육/육아 혜택으로 더 많은 즐거움을 누리세요. KB국민 와이즈카드, 당신의 행복을 담아보세요."
3. "일상이 더 즐거워지는 순간! 🛬 카드 혜택으로 여유롭게 여행을 즐기세요. KB국민 와이즈카드, 당신의 여행을 더해 보세요."

이 문구들은 카드의 주요 혜택을 강조하고, 감성적 요소인 여유로운 삶, 더 많은 즐거움, 여유롭게 여행을 즐기기 등이 포함되어 있습니다. 또한, 이모티콘을 사용하여 생동감을 더해 카드의 매력을 전달하고자 했습니다. 


# 6. 파인튜닝 : SFTTrainer

In [50]:
pip install git+https://github.com/huggingface/trl.git

  Cloning https://github.com/huggingface/trl.git to /tmp/pip-req-build-49ahxfh_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-req-build-49ahxfh_
  Resolved https://github.com/huggingface/trl.git to commit 066fc37bd3381e5de3ac0bb988ce834a050c459f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.7 MB/s eta 0:00:00
  Created wheel for trl: filename=trl-0.13.0.dev0-py3-none-any.whl size=313541 sha256=b47a1983050754ce1a1f448c7ecfd2cf4f030e70c8a418bc9b474b1564b30340
  Stor

In [55]:
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

# 1. 모델 및 토크나이저 로드
model_path = "allganize/Llama-3-Alpha-Ko-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype="bfloat16")

tokenizer = AutoTokenizer.from_pretrained(model_path)

# 2. 데이터 로드
data_file_path = "/content/diversified_card_with_ad_copies.jsonl"
dataset = load_dataset("json",data_files={"train":data_file_path})
train_dataset = dataset["train"]

print(train_dataset[0])

# 3. SFTTrainer 설정
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    # max_seq_length=512,
    per_device_train_batch_size=2,
    gradient_accmulation_steps=8,
    num_train_epochs=3,
    output_dir="./llama-copywrite-finetuned",
    fp16=True, # 혼합 정밀도
)

# 4. Fine-Tuning 실행
trainer.train()


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

{'Card Name': '하나 트래블로그 SKYPASS 신용카드', 'Annual_fee': '국내 4만8천원, 해외 4만8천원, 가족겸용 없음', 'Base Record': '조건없음', 'Category_List': "['항공마일리지']", 'Ad Copy 1': '"여행을 꿈꾸신다면? 하나 트래블로그 SKYPASS 신용카드 ✈️로 마일리지 쌓고, 하늘로 떠나요!"', 'Ad Copy 2': '"하나 트래블로그 SKYPASS 신용카드로 누리는 특별한 하루, 지금 바로 시작해보세요! 🌟"', 'Ad Copy 3': '"하나 트래블로그 SKYPASS 신용카드로 누리는 특별한 하루, 지금 바로 시작해보세요! 🌟"'}


TypeError: SFTTrainer.__init__() got an unexpected keyword argument 'per_device_train_batch_size'

In [ ]:
# 5.